In [16]:
from glob import glob 
from pathlib import Path
from sklearn.model_selection import train_test_split 
import nibabel as nib 
import numpy as np 

In [6]:
#list volumes and list labels in the directories 
source_dir =Path("/media/Datacenter_storage/ramon_dataset_curations/kidney_segmentation/source/kits_data_spec/kits21/data")
label_dir =Path("/media/Datacenter_storage/ramon_dataset_curations/kidney_segmentation/source/kits_data_spec/kits21/data")
image_paths = list(source_dir.rglob("*imaging*"))
label_paths = list(label_dir.rglob("*aggregated_AND_seg.nii.gz"))

In [11]:
# parse their ids based on name 
def gen_dict(dir_iter):
    out_dict = dict() 
    print(len(dir_iter))
    for e in dir_iter:
        f_path = str(e)
        case_name = f_path.split('/')[-2]
        out_dict[case_name] = f_path
    return out_dict
image_dict = gen_dict(image_paths)
label_dict = gen_dict(label_paths)

300
300


In [12]:

seg_samples = list() 
for e in image_dict.keys(): 
    sample_dict = dict() 
    sample_dict['image'] = image_dict[e]
    sample_dict['label']= label_dict[e]
    sample_dict['phase']= 1
    seg_samples.append(sample_dict)


In [10]:

#this is an old processing step that would   modify masks to be kidney only 
""" 
for e in seg_samples:
    vol = nib.load(e['label'])
    arr = (vol.get_fdata() ==1).astype(np.int)
    nii =nib.Nifti1Image(arr,affine=vol.affine,header=vol.header)
    nib.save(nii,e['label'])
"""

" \nfor e in seg_samples:\n    vol = nib.load(e['label'])\n    arr = (vol.get_fdata() ==1).astype(np.int)\n    nii =nib.Nifti1Image(arr,affine=vol.affine,header=vol.header)\n    nib.save(nii,e['label'])\n"

In [14]:
# sort the files by name to have them in a consistent ordering 
seg_samples = sorted(seg_samples,key= lambda x: x['image'])

In [17]:

train,test = train_test_split(seg_samples,random_state=1996)
test,val = train_test_split(test,random_state=1996)

In [18]:

kits21 = (train,test,val)

In [19]:
import pickle  as pkl 

with open('../data/kits21_data.pkl','wb') as f: 
    pkl.dump(kits21,f) 

FileNotFoundError: [Errno 2] No such file or directory: '../data/kits21_data.pkl'